In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras

import os
from matplotlib import pyplot as plt
import cv2

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/eurecom-aml-2021-challenge-2/sample_submission.csv
/kaggle/input/eurecom-aml-2021-challenge-2/refuge_data/refuge_data/val/index.json
/kaggle/input/eurecom-aml-2021-challenge-2/refuge_data/refuge_data/val/gts/V0399.bmp
/kaggle/input/eurecom-aml-2021-challenge-2/refuge_data/refuge_data/val/images/V0190.jpg
/kaggle/input/eurecom-aml-2021-challenge-2/refuge_data/refuge_data/test/index.json
/kaggle/input/eurecom-aml-2021-challenge-2/refuge_data/refuge_data/test/gts/T0037.bmp
/kaggle/input/eurecom-aml-2021-challenge-2/refuge_data/refuge_data/test/images/T0247.jpg
/kaggle/input/eurecom-aml-2021-challenge-2/refuge_data/refuge_data/train/index.json
/kaggle/input/eurecom-aml-2021-challenge-2/refuge_data/refuge_data/train/gts/n0222.bmp
/kaggle/input/eurecom-aml-2021-challenge-2/refuge_data/refuge_data/train/images/n0166.jpg
/kaggle/input/eurecom-aml-2021-challenge-2/refuge_data/refuge_data/train/illustrations/n0166.jpg


In [2]:
def remove_nerves(image):
    img = tf.keras.utils.array_to_img(image)
    # fig = plt.figure(figsize=(50,20))
    # plt.title('Original'); plt.imshow(img); plt.show()
    orig_size = np.array(img).shape[:2]
    orig_size = (orig_size[1], orig_size[0])
    # print(orig_size)
    img = cv2.resize(np.array(img),(1024,1024))
    img = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2RGB)
    # plt.subplot(1, 5, 1)
    # plt.imshow(img)
    # plt.axis('off')
    # plt.title('Original')

    # convert image to grayScale
    grayScale = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # plt.subplot(1, 5, 2)
    # plt.imshow(grayScale)
    # plt.axis('off')
    # plt.title('GrayScale')

    # kernel for morphologyEx
    kernel = cv2.getStructuringElement(1,(17,17))


    # apply MORPH_BLACKHAT to grayScale image
    blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)
    # plt.subplot(1, 5, 3)
    # plt.imshow(blackhat)
    # plt.axis('off')
    # plt.title('blackhat')

    # apply thresholding to blackhat
    _,threshold = cv2.threshold(blackhat,10,255,cv2.THRESH_BINARY)
    # plt.subplot(1, 5, 4)
    # plt.imshow(threshold)
    # plt.axis('off')
    # plt.title('threshold')

    # inpaint with original image and threshold image
    final_image = cv2.inpaint(img,threshold,1,cv2.INPAINT_TELEA)
    # plt.subplot(1, 5, 5)
    # plt.imshow(cv2.cvtColor(final_image, cv2.COLOR_BGR2RGB))
    # plt.axis('off')
    # plt.title('final_image')

    # plt.plot()
    final_image = cv2.cvtColor(final_image, cv2.COLOR_BGR2RGB)
    final_image = cv2.resize(final_image,orig_size)
    # plt.title('No Nerve'); plt.imshow(final_image); plt.show()
    return final_image#.astype(np.float64)/255.0

In [3]:
TRAIN_DIR = '/kaggle/input/eurecom-aml-2021-challenge-2/refuge_data/refuge_data/train/'
os.listdir(TRAIN_DIR)

['gts', 'images', 'index.json', 'illustrations']

In [4]:
images = os.path.join(TRAIN_DIR, 'images')
masks = os.path.join(TRAIN_DIR, 'gts')

In [5]:
from PIL import Image
def crop_image(image_path, mask_path, remove_nerves=False):
    image = cv2.cvtColor(cv2.imread(os.path.join(images, image_path)), cv2.COLOR_BGR2RGB)
    raw_mask = np.array(Image.open(os.path.join(masks, mask_path), mode='r'))
    mask = raw_mask.copy()
    raw_mask[raw_mask == 128] = 1
    raw_mask[raw_mask == 0] = 2
    raw_mask[raw_mask == 255] = 0

    
    # invert mask colors
    mask[mask == 255] = 100
    mask[mask == 128] = 0
    mask[mask == 0] = 1
    mask[mask == 100] = 0
    ret,thresh = cv2.threshold(mask,0, 1, 0)
    contours,hierarchy = cv2.findContours(thresh, 1, 2)
    cnt = contours[0]
    x,y,w,h = cv2.boundingRect(cnt)
    return image, raw_mask
    

In [6]:
save_dir = "./train_cropped"
!mkdir "train_cropped"
!mkdir "train_cropped/images"
!mkdir "train_cropped/masks"

for ind, f in enumerate(os.listdir(images)):
    fname = f[:-4]
    image, mask = crop_image(fname + ".jpg", fname + ".bmp")
    #print(ind)
    msk = Image.fromarray(mask.astype('uint8'), 'L')
    msk.save(os.path.join(save_dir + "/masks", fname + '.png'))
#     im = Image.fromarray(image)
#     im.save(os.path.join(save_dir + "/images", fname + '.jpg'))

In [7]:
%cd /kaggle/working/train_cropped
%ls images/

/kaggle/working/train_cropped


In [8]:
%cd /kaggle/working/train_cropped
from IPython.display import FileLinks
#FileLinks(r'masks')

/kaggle/working/train_cropped
